<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/Theo/Dacon_satellite_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tensorflow.keras.layers import LeakyReLU
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
 
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 1. Data collection and preprocessing

In [0]:
# 1.1 Data Collection
# train
train_origin = np.load('/content/drive/My Drive/2020 Kaggle Study/data/yeonung/train.npy')[:,:,:,:12]

# feature
train_temp = train_origin[:,:,:,:9]

train_landtype = train_origin[:,:,:,9]
train_lon_GMI = train_origin[:,:,:,10]
train_lat_GMI = train_origin[:,:,:,11]
train_DEM = np.load("/content/drive/My Drive/2020 Kaggle Study/data/DEM_DPR/dem_train_DPR.npy")

# target
train_rain = np.load("/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci_32bits.npy")

# test
test_origin = np.load('/content/drive/My Drive/2020 Kaggle Study/data/yeonung/test.npy')[:,:,:,:12]

# feature
test_temp = test_origin[:,:,:,:9]

test_landtype = test_origin[:,:,:,9]
test_lon_GMI = test_origin[:,:,:,10]
test_lat_GMI = test_origin[:,:,:,11]
test_DEM = np.load("/content/drive/My Drive/2020 Kaggle Study/data/DEM_DPR/dem_test_DPR.npy")

del train_origin

all_temp = np.concatenate((train_temp, test_temp), axis = 0)
del train_temp, test_temp
all_landtype = np.concatenate((train_landtype, test_landtype), axis = 0)
del train_landtype, test_landtype
all_lon_GMI = np.concatenate((train_lon_GMI, test_lon_GMI), axis = 0)
del train_lon_GMI, test_lon_GMI
all_lat_GMI = np.concatenate((train_lat_GMI, test_lat_GMI), axis = 0)
del train_lat_GMI, test_lat_GMI
all_DEM = np.concatenate((train_DEM, test_DEM), axis = 0)
del train_DEM, test_DEM

In [5]:
all_temp.shape

(78761, 40, 40, 9)

In [0]:
# 1.2 Data preprocessing
def minmaxscaler(data_info, n):
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  data_in = data_info.reshape(-1, 1)
  scaler.fit(data_in)
  data_in = scaler.transform(data_in)
  data_in = data_in.reshape(-1)
  data_info = data_in.reshape(n, 40, 40, 1)
  return data_info

def standardscaler(data_info):
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  data_in = data_info.reshape(-1, 1)
  scaler.fit(data_in)
  data_in = scaler.transform(data_in)
  data_in = data_in.reshape(-1)
  data_info = data_in.reshape(78761, 40, 40, 1)
  return data_info

In [7]:
# temp - standard , land - categorization, dem - minmax, rain - original
print("temp")
for i in range(0,9):
  t_temp = standardscaler(all_temp[:,:,:,i])
  print("cat : %d" %i)
  all_temp[:,:,:,i:i+1] = t_temp

print("land")
t_land = all_landtype.reshape(-1)
for i in range(len(t_land)):
  t_land[i] = int(str(t_land[i])[0])
all_landtype = t_land.reshape(78761,40, 40, 1)

print("DEM")
all_DEM = minmaxscaler(all_DEM, 78761)

print("GMI lon, lat")
all_lon_GMI = standardscaler(all_lon_GMI)
all_lat_GMI = standardscaler(all_lat_GMI)

#print("train rain")
train_rain = np.where(train_rain>0, train_rain, 0)
#train_rain = minmaxscaler(train_rain, 76345)

temp
cat : 0
cat : 1
cat : 2
cat : 3
cat : 4
cat : 5
cat : 6
cat : 7
cat : 8
land
DEM
GMI lon, lat


In [0]:
train = np.concatenate((all_temp[:76345,:,:,:], all_landtype[:76345,:,:,:], all_DEM[:76345,:,:,:], all_lon_GMI[:76345,:,:,:], all_lat_GMI[:76345,:,:,:], train_rain), axis = 3)
test = np.concatenate((all_temp[76345:,:,:,:], all_landtype[76345:,:,:,:], all_DEM[76345:,:,:,:], all_lon_GMI[76345:,:,:,:], all_lat_GMI[76345:,:,:,:]), axis = 3)

In [0]:
del all_temp, all_landtype, all_DEM, all_lon_GMI, all_lat_GMI, train_rain

In [0]:
train = tf.convert_to_tensor(train, dtype=np.float32)
train = train.numpy()
test = tf.convert_to_tensor(test, dtype=np.float32)
test = test.numpy()

In [11]:
print(train.shape)
print(test.shape)

(76345, 40, 40, 14)
(2416, 40, 40, 13)


## 2. 모델 생성

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

In [0]:
def build_model(input_layer_c, input_layer_d, start_neurons):

    # 40 x 40 -> 20 x 20
    convc1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(input_layer_c)
    convc1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convc1)
    poolc1 = BatchNormalization()(convc1)
    poolc1 = MaxPooling2D((2, 2))(poolc1)
    poolc1 = Dropout(0.25)(poolc1)
    # 20 x 20 -> 10 x 10
    convc2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poolc1)
    convc2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convc2)
    poolc2 = BatchNormalization()(convc2)
    poolc2 = MaxPooling2D((2, 2))(poolc2)
    poolc2 = Dropout(0.25)(poolc2)
    # 10 x 10 -> 5 x 5
    convc3 = Conv2D(start_neurons * 4, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poolc2)
    convc3 = Conv2D(start_neurons * 4, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convc3)
    poolc3 = BatchNormalization()(convc3)
    poolc3 = MaxPooling2D((2, 2))(poolc3)
    poolc3 = Dropout(0.25)(poolc3)

    # 5 x 5
    convmc = Conv2D(start_neurons * 8, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poolc3)
    convmc = BatchNormalization()(convmc)

    # 40 x 40 -> 20 x 20
    convd1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(input_layer_d)
    convd1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convd1)
    poold1 = BatchNormalization()(convd1)
    poold1 = MaxPooling2D((2, 2))(poold1)
    poold1 = Dropout(0.25)(poold1)
    # 20 x 20 -> 10 x 10
    convd2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poold1)
    convd2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convd2)
    poold2 = BatchNormalization()(convd2)
    poold2 = MaxPooling2D((2, 2))(poold2)
    poold2 = Dropout(0.25)(poold2)
    # 10 x 10 -> 5 x 5
    convd3 = Conv2D(start_neurons * 4, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poold2)
    convd3 = Conv2D(start_neurons * 4, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convd3)
    poold3 = BatchNormalization()(convd3)
    poold3 = MaxPooling2D((2, 2))(poold3)
    poold3 = Dropout(0.25)(poold3)
    # 5 x 5
    convmd = Conv2D(start_neurons * 8, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(poold3)
    convmd = BatchNormalization()(convmd)

    convm = concatenate([convmc, convmd], axis = 3)
    convm = BatchNormalization()(convm)

    # 5 x 5 -> 10 x 10
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(convm)
    uconv3 = concatenate([deconv3, convc3, convd3])
    uconv3 = Dropout(0.25)(uconv3)
    uconv3 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv3)
    uconv3 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv3)
    uconv2 = concatenate([deconv2, convc2, convd2])
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same", activation = tf.keras.layers.LeakyReLU(alpha=0.1))(uconv2)
    uconv1 = concatenate([deconv1, convc1, convd1])
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1),activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same", kernel_initializer='glorot_uniform')(uconv1)
    
    return output_layer


In [0]:
input_layer_c = Input((40, 40, 9))
input_layer_d = Input((40, 40, 4))
output_layer = build_model(input_layer_c, input_layer_d,  32)

In [0]:
model = Model(inputs = [input_layer_c, input_layer_d], outputs = [output_layer])

In [0]:
from sklearn.metrics import f1_score
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))
def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))
def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)
def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score
def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="mae", optimizer=opt, metrics=[maeOverFscore_keras, fscore_keras])

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True)
model_history = []
fold_number = 0

for train_index, test_index in kf.split(train):
    print(f'\nFold {fold_number}')
    fold_number = fold_number + 1
    
#-----------------------------------------------------------------------------
    def trainGenerator():
        print(1)
        for file in train[train_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:13]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels

    train_dataset = tf.data.Dataset.from_generator(trainGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 4])), tf.TensorShape([40,40,1])))
    
    train_dataset = train_dataset.batch(512).prefetch(1)
#-----------------------------------------------------------------------------
    def testGenerator():
        print(2)
        for file in train[test_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:13]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels
    test_dataset = tf.data.Dataset.from_generator(testGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 4])),tf.TensorShape([40,40,1])))
    test_size = len(test_index)
    test_dataset = test_dataset.batch(test_size).prefetch(1)
#-----------------------------------------------------------------------------
    history = model.fit(train_dataset, epochs = 20, verbose=1, validation_data = test_dataset)
    model.save("/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_attention.h5")
    #history = model.fit(train_dataset, epochs = 5, ,batch_size = 11, verbose=1)
    model_history.append(history)
    #del data
    del train_dataset
    del test_dataset
    gc.collect()


Fold 0
Epoch 1/20
1
     77/Unknown - 241s 3s/step - loss: 0.2407 - maeOverFscore_keras: 2.8778 - fscore_keras: 0.54252
77/77 [==============================] - 271s 4s/step - loss: 0.2407 - maeOverFscore_keras: 2.8778 - fscore_keras: 0.5425 - val_loss: 0.1814 - val_maeOverFscore_keras: 2.1954 - val_fscore_keras: 0.6243
Epoch 2/20
1
77/77 [==============================] - 280s 4s/step - loss: 0.1757 - maeOverFscore_keras: 1.8922 - fscore_keras: 0.6637 - val_loss: 0.2009 - val_maeOverFscore_keras: 1.8081 - val_fscore_keras: 0.6818
Epoch 3/20
1
77/77 [==============================] - ETA: 0s - loss: 0.1654 - maeOverFscore_keras: 1.7240 - fscore_keras: 0.69392


In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5', custom_objects={"maeOverFscore_keras": maeOverFscore_keras,"LeakyReLU":LeakyReLU, "fscore_keras":fscore_keras })

In [0]:
#test_tempandland = np.load("/content/drive/My Drive/2020 Kaggle Study/data/yeonung/test.npy")[:,:,:,:10]
#test_DEM = np.load("/content/drive/My Drive/2020 Kaggle Study/data/DEM_DPR/dem_test_DPR.npy")
#test_day = np.load("/content/drive/My Drive/2020 Kaggle Study/data/orbit_no_shuffle/test_days.npy")
#test_times = np.load("/content/drive/My Drive/2020 Kaggle Study/data/orbit_no_shuffle/test_times.npy")

In [0]:
#test = np.concatenate((test_tempandland, test_DEM, test_day, test_times), axis = 3)

In [15]:
print(test.shape)

(2416, 40, 40, 13)


In [0]:
pred=model.predict([test[:,:,:,0:9], test[:,:,:,9:]])
pred = np.where(pred>0, pred, 0)

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')

In [0]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [0]:
submission.to_csv('/content/drive/My Drive/2020 Kaggle Study/Theos/Submission/three_leg_CNN_second.csv', index = False)